In [45]:
%reload_ext autoreload
%autoreload 2
import autohf as hf
import numpy as np
from matplotlib import pyplot as plt
import time
import autograd
import chemistry as chem
import autograd.numpy as anp
import openfermion
from pennylane import qchem
from tqdm import tqdm
import pennylane as qml

# Optimizing a Basis Set Using AutoHF

In [33]:
M1, M2 = hf.generate_basis_set('sto-3g', ['H', 'H']) # Generates default information for hydrogen
num_elecs = 2
R1, R2 = np.array([0.0, 0.0, 0.0]), np.array([1.0, 0.0, 0.0])
R = list(R1) + list(R2)

AO = []
initial_exp = []

# Generates atomic orbitals using the default information
for func in M1:
    L, exp, coeff = func
    initial_exp.extend(exp)
    AO.append(hf.AtomicBasisFunction(L, C=coeff, R=R1))

# Generates atomic orbitals using the default information
for func in M2:
    L, exp, coeff = func
    initial_exp.extend(exp)
    AO.append(hf.AtomicBasisFunction(L, C=coeff, R=R2))

In [34]:
one_elec_func = lambda idx, *args : hf.one_electron_integral(2, AO, idx)([args[0:3]], [args[3:6]])
two_elec_func = lambda idx, *args : hf.two_electron_integral(2, AO, idx)([args[0:3]], [args[3:6]])

In [35]:
# Function to generate all one electron derivatives
def generate_one_electron(func, A, idx=None):
    
    num_atoms = int(len(A) / 3)
    one_electron_derivatives = [[0 for i in range(num_atoms)] for j in range(num_atoms)]
    
    for i in range(num_atoms):
        for j in range(num_atoms):
            if i <= j:
                generator = lambda x1, y1, z1, x2, y2, z2 : func([i, j], x1, y1, z1, x2, y2, z2)
                grad_func = autograd.grad(generator, argnum=idx)
                gradient = grad_func(*A)
            
                one_electron_derivatives[i][j] = gradient
                one_electron_derivatives[j][i] = gradient
    return np.array(one_electron_derivatives)

In [36]:
# Function to generate all two electron derivatives
def generate_two_electron(func, A, N=1, idx=None):
    
    num_atoms = int(len(A) / 3)
    two_electron_derivatives = [[[[0 for h in range(num_atoms)] for i in range(num_atoms)] for j in range(num_atoms)] for k in range(num_atoms)]

    for h in range(num_atoms):
        for i in range(num_atoms):
            for j in range(num_atoms):
                for k in range(num_atoms):
                    if h <= i and j <= k and hf.dict_ord((h, i), (j, k)):
                        generator = lambda x1, y1, z1, x2, y2, z2: func([h, i, j, k], x1, y1, z1, x2, y2, z2)
                        grad_func = autograd.grad(generator, argnum=idx)
                        gradient = grad_func(*A)

                        two_electron_derivatives[h][i][j][k] = gradient
                        two_electron_derivatives[h][i][k][j] = gradient
                        two_electron_derivatives[i][h][j][k] = gradient
                        two_electron_derivatives[i][h][k][j] = gradient

                        two_electron_derivatives[j][k][h][i] = gradient
                        two_electron_derivatives[k][j][h][i] = gradient
                        two_electron_derivatives[j][k][i][h] = gradient
                        two_electron_derivatives[k][j][i][h] = gradient
    return np.array(two_electron_derivatives)

In [37]:
def build_h_from_integrals(geometry, one_electron, two_electron, nuc_energy, wires, basis="sto-3g", multiplicity=1, charge=0):
    molecule = openfermion.MolecularData(geometry=geometry, basis=basis, multiplicity=multiplicity, charge=charge)
    molecule.one_body_integrals = one_electron
    molecule.two_body_integrals = two_electron
    
    molecule.nuclear_repulsion = nuc_energy
    
    H = molecule.get_molecular_hamiltonian()
    fermionic_hamiltonian = openfermion.transforms.get_fermion_operator(H)
    o = openfermion.transforms.jordan_wigner(fermionic_hamiltonian)
    ham = qchem.convert_observable(o, wires=wires)
    return ham

In [38]:
def nuc_energy(R):
    x1, y1, z1, x2, y2, z2 = R[0], R[1], R[2], R[3], R[4], R[5]
    
    e1 = 1 / anp.sqrt((x1 - x2) **2 + (y1 - y2) ** 2 + (z1 - z2) ** 2)
    return e1

nuc = nuc_energy(np.array(R))
nuc_d = autograd.jacobian(nuc_energy)(np.array(R))

In [39]:
bohr_angs = 0.529177210903
wires = [0, 1, 2, 3]
new_coordinates = bohr_angs * np.array([R[0:3], R[3:6]])

In [40]:
def H(alpha):
    one_elec, two_elec = hf.electron_integrals(num_elecs, AO)([alpha[0:3]], [alpha[3:6]])
    H0 = build_h_from_integrals(list(zip(['H', 'H'], new_coordinates)), one_elec, two_elec, nuc, wires)
    return H0

In [41]:
def grad_H(alpha):
    H1 = []
    for i in tqdm(range(len(nuc_d))):
        H1.append(build_h_from_integrals(list(zip(['H', 'H'], new_coordinates)), generate_one_electron(one_elec_func, alpha, idx=i), generate_two_electron(two_elec_func, alpha, idx=i), nuc_d[i], wires))
    return H1

In [49]:
# Calculates the molecular Hessian

num_params = 1 # Number of variational parameters
qubits = 4 # Number of qubits

dev = qml.device('default.qubit', wires=qubits) # Defines the device used

# Creates the circuit used to construct the VQE function
def circuit(params, **kwargs):
    qml.BasisState(np.array([1, 1, 0, 0]), wires=dev.wires)
    qml.DoubleExcitation(params[0], wires=[0, 1, 2, 3])
    
optimizers = (qml.GradientDescentOptimizer(stepsize=0.4), qml.GradientDescentOptimizer(stepsize=0.4))
initial_params = (np.array([0.0]), np.array(initial_exp))
steps = 20

# Performs VQE
energy, params, opt_exp = chem.analytic_geometry(H, grad_H, circuit, dev, optimizers, steps, initial_params, bar=True)

Energy = -1.065999461556561, Geometry = 3.425250914:   0%| | 0/20 [00:00<?
100%|███████████████████████████████████████| 6/6 [01:32<00:00, 15.37s/it]
Energy = -1.0758751397763928, Geometry = 3.0260234132276635:   5%| | 1/20 
100%|███████████████████████████████████████| 6/6 [02:51<00:00, 28.51s/it]
Energy = -1.0760383933631437, Geometry = 2.628267773544274:  10%| | 2/20 [
100%|███████████████████████████████████████| 6/6 [03:41<00:00, 36.88s/it]
Energy = -1.0714426886700914, Geometry = 2.2322374299580896:  15%|▏| 3/20 
  0%|                                               | 0/6 [00:20<?, ?it/s]
Energy = -1.0714426886700914, Geometry = 2.2322374299580896:  15%|▏| 3/20 


KeyboardInterrupt: 